In [3]:
import os
import sys

# Configure the environment
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = 'C:\spark-1.4.1-bin-hadoop2.4\spark-1.4.1-bin-hadoop2.4'

# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

# Add the PySpark/py4j to the Python Path
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "lib/py4j-0.8.2.1-src.zip"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))
execfile(os.path.join(SPARK_HOME, 'python/pyspark/shell.py'))

IOError: [Errno 2] No such file or directory: 'C:/spark/spark-1.4.1-bin-hadoop2.4\\python/pyspark/shell.py'

In [2]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
conf = SparkConf() 
(conf.setMaster("local[4]").setAppName("KeyValuePair").set("spark.driver.memory", "2G"))
sc = SparkContext(conf=conf)

ImportError: No module named pyspark

In [3]:
left=[(1, 'A', 'B' ),(2, 'C', 'D' ),(2, 'E', 'F' ),(3, 'E', 'F' )] 
left=sc.parallelize(left)

right=[(1, 'Z', 'Y' ),(1, 'X', 'V' ),(2, 'W', 'U' ),(4, 'T', 'S' )]
right=sc.parallelize(right)

In [4]:
left = left.keyBy(lambda r: r[0])
right = right.keyBy(lambda r: r[0])
left.collect()

[(1, (1, 'A', 'B')),
 (2, (2, 'C', 'D')),
 (2, (2, 'E', 'F')),
 (3, (3, 'E', 'F'))]

In [5]:
left = left.map(lambda (k, v): (k, (1, v)))
right = right.map(lambda (k, v): (k, (2, v)))
left.collect()

[(1, (1, (1, 'A', 'B'))),
 (2, (1, (2, 'C', 'D'))),
 (2, (1, (2, 'E', 'F'))),
 (3, (1, (3, 'E', 'F')))]

In [6]:
unioned = left.union(right)
unioned.collect()

[(1, (1, (1, 'A', 'B'))),
 (2, (1, (2, 'C', 'D'))),
 (2, (1, (2, 'E', 'F'))),
 (3, (1, (3, 'E', 'F'))),
 (1, (2, (1, 'Z', 'Y'))),
 (1, (2, (1, 'X', 'V'))),
 (2, (2, (2, 'W', 'U'))),
 (4, (2, (4, 'T', 'S')))]

In [7]:
grouped = unioned.groupByKey()
result = grouped.flatMapValues(lambda x :x)
result.collect()

[(1, (1, (1, 'A', 'B'))),
 (1, (2, (1, 'Z', 'Y'))),
 (1, (2, (1, 'X', 'V'))),
 (2, (1, (2, 'C', 'D'))),
 (2, (1, (2, 'E', 'F'))),
 (2, (2, (2, 'W', 'U'))),
 (3, (1, (3, 'E', 'F'))),
 (4, (2, (4, 'T', 'S')))]

In [8]:
def dispatch(seq):
    left_origin = []
    right_origin = []
    for (n, v) in seq:
        if n == 1:
            left_origin.append(v)
        elif n == 2:
            right_origin.append(v)
    return [(v, w) for v in left_origin for w in right_origin]

In [9]:
result = grouped.flatMapValues(lambda x :dispatch(x))

In [10]:
result.collect()

[(1, ((1, 'A', 'B'), (1, 'Z', 'Y'))),
 (1, ((1, 'A', 'B'), (1, 'X', 'V'))),
 (2, ((2, 'C', 'D'), (2, 'W', 'U'))),
 (2, ((2, 'E', 'F'), (2, 'W', 'U')))]

In [11]:
data = [(1, 'a'), (1, 'b'), (2, 'c'), (2, 'd'), (2, 'e'), (3, 'f')]
rdd = sc.parallelize(data)
result = rdd.groupByKey().collect()

In [13]:
x = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
map((lambda (x,y): (x, list(y))), sorted(x.groupByKey().collect()))

[('a', [1, 1]), ('b', [1])]

In [14]:
x = sc.parallelize([("a", ["x", "y", "z"]), ("b", ["p", "r"])])
def f(x): 
    return x
x.flatMapValues(f).collect()

[('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]

In [15]:
x = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
def f(x): 
    return len(x)
x.mapValues(f).collect()

[('a', 3), ('b', 1)]

In [1]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
from operator import add
rdd.foldByKey(1, add).collect()

NameError: name 'sc' is not defined

In [17]:
#Generic function to combine the elements for each key using a custom set of aggregation functions.

#Turns an RDD[(K, V)] into a result of type RDD[(K, C)], for a "combined type" C. 
#Note that V and C can be different -- 
#for example, one might group an RDD of type (Int, Int) into an RDD of type (Int, List[Int]).

#Users provide three functions:

#createCombiner, which turns a V into a C (e.g., creates a one-element list)
#mergeValue, to merge a V into a C (e.g., adds it to the end of a list)
#mergeCombiners, to combine two C's into a single one.

x = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
def f(x): return x
def add(a, b): return a + str(b)
sorted(x.combineByKey(str, add, add).collect())

[('a', '11'), ('b', '1')]